In [1]:
import pandas as pd  # Importing pandas for data manipulation and analysis
import numpy as np  # Importing numpy for numerical operations and array handling
import paramiko  # Importing paramiko for SSH and SFTP operations
import json  # Importing json to work with JSON data (serialization and deserialization)
import os  # Importing os for interacting with the operating system (e.g., file paths, environment variables)
import io  # Importing io to handle in-memory byte streams
from paramiko import SSHClient  # Importing SSHClient specifically for managing SSH connections
import pyodbc as odbc  # Importing pyodbc for connecting to and interacting with databases via ODBC

In [2]:
with open(".env/config.json", 'r') as file:
    config = json.load(file)

hostname = config['SFTP_HOSTNAME']
port = config['SFTP_PORT']
username = config['SFTP_USERNAME']
password = config['SFTP_PASSWORD']
SERVER = config['SERVER']
DATABASE = config['SQL_DATABASE_PROD']
UID = config['SQL_UID']
PWD = config['SQL_PWD']

file.close()

In [6]:
# Define the directory path where the renewal data is stored
dir_path = '/opt/abanet/BI/Renewal_Data/'

# Establish SSH transport
transport = paramiko.Transport((hostname, port))
transport.connect(username=username, password=password)

# Establish SFTP session
sftp = paramiko.SFTPClient.from_transport(transport)

# Change directory to remote directory
sftp.chdir(dir_path)

# Get list of files in the directory
files = sftp.listdir()

print(len(files))

84


In [7]:
# Define the months and file categories
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_full_names = {
    'Jan': 'jan_23', 'Feb': 'feb_23', 'Mar': 'mar_23', 'Apr': 'apr_23',
    'May': 'may_23', 'Jun': 'jun_23', 'Jul': 'jul_23', 'Aug': 'aug_23',
    'Sep': 'sep_22', 'Oct': 'oct_22', 'Nov': 'nov_22', 'Dec': 'dec_22'
}
file_categories = ['Article', 'Books', 'Events-CLE', 'On_Demand_Video', 'News-ABA', 'Podcast', 'ABA_Advantage']

# Iterate through all 12 months and create separate DataFrames for each month
for month in months:
    # Dynamically create the file order for each month
    file_order = [f'{month}_{category}.csv' for category in file_categories]
    df_list = []

    # Loop through the files for the current month and read them
    for file_name, category in zip(file_order, file_categories):
        # Read the file into a pandas DataFrame
        with sftp.open(file_name, 'r') as f:
            file_content = f.read()  # Read the file content
            file_buffer = io.StringIO(file_content.decode())  # Decode and convert to buffer
            j_df = pd.read_csv(file_buffer)  # Read into DataFrame
            j_df.rename(columns={'ABA ID (eVar2) (evar2)': 'customer_id', 'Visits': category}, inplace=True)
            df_list.append(j_df)

    # Start with the first DataFrame in df_list
    monthly_df = df_list[0]

    # Iterate through the remaining DataFrames and merge them
    for i in range(1, len(df_list)):
        monthly_df = pd.merge(monthly_df, df_list[i], on='customer_id', how='outer')

    # Convert numeric columns to numeric, fill NaNs with 0, and convert to integers
    for col in monthly_df.columns:
        if col != 'customer_id':
            monthly_df[col] = pd.to_numeric(monthly_df[col], errors='coerce').fillna(0).astype(int)

    # Drop rows where 'Member_ID' is 0 or NaN
    monthly_df = monthly_df[monthly_df['customer_id'].notna()]
    monthly_df = monthly_df[monthly_df['customer_id'] != 0]

    # Ensure 'Member_ID' has 8 characters by adding leading zeros
    monthly_df['customer_id'] = monthly_df['customer_id'].astype(int).astype(str).str.zfill(8)

    # Assign the DataFrame to a variable named after the full month name
    globals()[f"{month_full_names[month]}_df"] = monthly_df
    #sftp.close()

In [8]:
# Get a list of all DataFrames in the global namespace
dataframes = [var for var in globals() if isinstance(globals()[var], pd.DataFrame)]

# Print the list of DataFrame names
print(dataframes)

['j_df', 'monthly_df', 'jan_23_df', 'feb_23_df', 'mar_23_df', 'apr_23_df', 'may_23_df', 'jun_23_df', 'jul_23_df', 'aug_23_df', 'sep_22_df', 'oct_22_df', 'nov_22_df', 'dec_22_df']


### Random Sampling to Validate the Adobe Data

In [ ]:
# List of dataframe variable names as strings, representing monthly data
dataframes = [
    'jan_23_df', 'feb_23_df', 'mar_23_df', 'apr_23_df', 'may_23_df', 
    'jun_23_df', 'jul_23_df', 'aug_23_df', 'sep_22_df', 'oct_22_df', 
    'nov_22_df', 'dec_22_df'
]

# Iterate over each dataframe name in the list
for df_name in dataframes:
    # Access the dataframe object using its name from the global namespace
    df = globals()[df_name]
    
    # Randomly sample 15 customer IDs from the 'customer_id' column, with a fixed random seed for reproducibility
    sample_id = df['customer_id'].sample(n=15, random_state=42)
    
    # Filter the dataframe to include only rows where 'customer_id' matches the sampled IDs
    sample_df = df.loc[df['customer_id'].isin(sample_id)]
    
    # Print the name of the dataframe being processed
    print(f"\nOutput for {df_name}:")
    # Print the filtered dataframe
    print(sample_df)

### Bringing in Personify Data

In [9]:
### Creating Monthly Cohorts ###
# Create a list of starting months
cohort_name = ["sep_22", "oct_22", "nov_22", "dec_22", "jan_23", "feb_23", "mar_23", 
               "apr_23", "may_23", "jun_23", "jul_23", "aug_23"]

start_months = pd.date_range("2022-09-01", periods=12, freq="MS")

# Initialize an empty list to store cohort start and end dates
cohorts = []

# Loop through the start months and calculate the corresponding end date for each cohort
for start_month in start_months:
    start_date = start_month
    end_date = start_month + pd.DateOffset(months=9, days=-1)  # 9 months minus 1 day
    
    # Store the cohort information
    cohorts.append({'cohort_start': start_date, 'cohort_end': end_date})

# Convert to DataFrame for easy visualization
cohorts_df = pd.DataFrame(cohorts)
cohorts_df['cohort_name'] = cohort_name
cohorts_df['cohort_start'] = cohorts_df['cohort_start'].dt.strftime('%Y-%m-%d')
cohorts_df['cohort_end'] = cohorts_df['cohort_end'].dt.strftime('%Y-%m-%d')
cohorts_df = cohorts_df[['cohort_name', 'cohort_start', 'cohort_end']]
cohorts_df.head()

,cohort_name,cohort_start,cohort_end
0,sep_22,2022-09-01,2023-05-31
1,oct_22,2022-10-01,2023-06-30
2,nov_22,2022-11-01,2023-07-31
3,dec_22,2022-12-01,2023-08-31
4,jan_23,2023-01-01,2023-09-30


In [10]:
# Construct the connection string for connecting to the SQL Server database. It includes the driver, server name, database name, user ID, and password
connection_string = "DRIVER={SQL SERVER}; SERVER=" + SERVER  + "; DATABASE=" + DATABASE + "; UID=" + UID + "; PWD=" + PWD + ";"
connection = odbc.connect(connection_string) # Establish a connection to the SQL Server database using the constructed connection string
# Loop through cohorts and query for each cohort
for index, cohort in cohorts_df.iterrows():
    cohort_name = cohort['cohort_name'] 
    cohort_start = cohort['cohort_start']
    cohort_end = cohort['cohort_end']
    
    # Construct the SQL query for the current cohort
    query = f"""
    SELECT
    SHIP_CUSTOMER.MASTER_CUSTOMER_ID customer_id,(( SUM(ORDER_DETAIL_MEASURES.REQUESTED_QTY) ) - ( SUM(ORDER_DETAIL_MEASURES.RETURNED_QTY) )) QUANTITY, ORDER_DETAIL.SUBSYSTEM,
    /*ORDER_MASTER      .ORDER_DATE SI 12/02/2020 TO sync with membership universe */
    ORDER_DETAIL.ORDER_DATE, PRODUCT_CLASS_APP_CODE.DESCR
    FROM
    ORDER_MASTER INNER JOIN ORDER_DETAIL ON (ORDER_DETAIL.ORDER_NO=ORDER_MASTER.ORDER_NO)
    RIGHT OUTER JOIN PRODUCT ON (ORDER_DETAIL.PRODUCT_ID=PRODUCT.PRODUCT_ID)
    INNER JOIN APP_CODE  PRODUCT_CLASS_APP_CODE ON (PRODUCT_CLASS_APP_CODE.TYPE = 'PRODUCT_CLASS'  AND  PRODUCT.PRODUCT_CLASS_CODE=PRODUCT_CLASS_APP_CODE.CODE
    AND PRODUCT.SUBSYSTEM = PRODUCT_CLASS_APP_CODE.SUBSYSTEM)
    INNER JOIN CUSTOMER  SHIP_CUSTOMER ON (SHIP_CUSTOMER.MASTER_CUSTOMER_ID=ORDER_DETAIL.SHIP_MASTER_CUSTOMER_ID and SHIP_CUSTOMER.SUB_CUSTOMER_ID=ORDER_DETAIL.SHIP_SUB_CUSTOMER_ID)
    INNER JOIN ORDER_DETAIL  ORDER_DETAIL_MEASURES ON (ORDER_DETAIL_MEASURES.ORDER_NO=ORDER_DETAIL.ORDER_NO and ORDER_DETAIL_MEASURES.ORDER_LINE_NO=ORDER_DETAIL.ORDER_LINE_NO)
    WHERE
    (
    /*ORDER_MASTER    .ORDER_DATE SI 12/02/2020 TO sync with membership universe */
    ORDER_DETAIL.ORDER_DATE  BETWEEN  '{cohort_start}'  AND  '{cohort_end}'
    AND
    ORDER_MASTER.ORDER_STATUS_CODE  =  'A'
    AND
    ORDER_DETAIL.LINE_STATUS_CODE  NOT IN  ( 'W','B','C','H','P','R'  )
    AND
    ORDER_DETAIL.SUBSYSTEM  <>  'MISC'
    )
    GROUP BY
    SHIP_CUSTOMER.MASTER_CUSTOMER_ID, 
    ORDER_DETAIL.SUBSYSTEM, 
    /*ORDER_MASTER    .ORDER_DATE SI 12/02/2020 TO sync with membership universe */
    ORDER_DETAIL.ORDER_DATE, 
    PRODUCT_CLASS_APP_CODE.DESCR
    """
    
    # odbc to execute the query and read it into a pandas DataFrame
    df_cohort = pd.read_sql(query, connection)
    globals()[f"order_{cohort_name}"] = df_cohort

C:\Users\SinghS\AppData\Local\Temp\1\ipykernel_21224\1141414625.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cohort = pd.read_sql(query, connection)


In [11]:
# List of months for which data will be processed
months = ['sep_22', 'oct_22', 'nov_22', 'dec_22', 'jan_23', 'feb_23', 'mar_23', 'apr_23', 'may_23', 'jun_23', 'jul_23', 'aug_23']

# Iterate over each month in the list
for month in months:
    # Construct the name of the DataFrame for the current month (e.g., 'order_sep_22')
    df_name = f"order_{month}"
    
    # Retrieve the DataFrame object from the global namespace using its name
    df = globals()[df_name]
    
    # Pivot the DataFrame to create one row per 'customer_id', with columns for each 'DESCR' value
    # The values in the pivot table are the sum of 'QUANTITY', with missing values filled with 0
    pivot_df = df.pivot_table(index='customer_id', columns='DESCR', values='QUANTITY', aggfunc='sum', fill_value=0)
    
    # Reset the index of the pivoted DataFrame to make 'customer_id' a regular column
    pivot_df.reset_index(inplace=True)
    
    # Construct the name for the pivoted DataFrame (e.g., 'sep_22_pivot')
    pivot_df_name = f"{month}_pivot"
    
    # Store the pivoted DataFrame back into the global namespace with the constructed name
    globals()[pivot_df_name] = pivot_df

In [12]:
# Set pandas display options for scrollable DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Prevent wrapping of columns
pd.set_option('display.max_rows', None)  # Show all rows

In [13]:
# Identify duplicate rows in the 'sep_22_pivot' DataFrame based on the 'customer_id' column
duplicates = sep_22_pivot[sep_22_pivot.duplicated(subset=['customer_id'], keep=False)] # 'keep=False' marks all duplicates as True, not just the subsequent ones

# Check if the 'duplicates' DataFrame is not empty (i.e., duplicates exist)
if not duplicates.empty:
    # Print a message indicating duplicates were found
    print("Duplicates found:")
    # Print the rows containing duplicate 'customer_id' values
    print(duplicates)

In [14]:
# Create a list of month names corresponding to the renewal periods
renewal_month_name = ["sep_22", "oct_22", "nov_22", "dec_22", "jan_23", "feb_23", "mar_23", 
               "apr_23", "may_23", "jun_23", "jul_23", "aug_23"]

# Generate a range of 12 monthly start dates beginning from September 1, 2023

renewal_start_months = pd.date_range("2023-09-01", periods=12, freq="MS") # 'freq="MS"' ensures the dates are the first day of each month

# Initialize an empty list to store dictionaries containing cohort start and end dates
renewals = []

# Loop through each start month in the generated date range
for renewal_month in renewal_start_months:
    # Set the start date of the cohort to the current month
    renewal_start_date = renewal_month
    # Calculate the end date of the cohort as the last day of the current month
    renewal_end_date = renewal_month + pd.DateOffset(months=1, days=-1)  # Add 1 month, subtract 1 day
    
    # Append a dictionary with the start and end dates to the renewals list
    renewals.append({'renewal_start': renewal_start_date, 'renewal_end': renewal_end_date})

# Convert the list of dictionaries into a DataFrame for easier manipulation and visualization
renewals_df = pd.DataFrame(renewals)

# Add the corresponding month names to the DataFrame
renewals_df['renewal_month_name'] = renewal_month_name

# Format the 'renewal_start' and 'renewal_end' columns as strings in 'YYYY-MM-DD' format
renewals_df['renewal_start'] = renewals_df['renewal_start'].dt.strftime('%Y-%m-%d')
renewals_df['renewal_end'] = renewals_df['renewal_end'].dt.strftime('%Y-%m-%d')

# Reorder the columns to display month name, start date, and end date in the desired order
renewals_df = renewals_df[['renewal_month_name', 'renewal_start', 'renewal_end']]

# Display the first few rows of the DataFrame to verify the results
renewals_df.head()

,renewal_month_name,renewal_start,renewal_end
0,sep_22,2023-09-01,2023-09-30
1,oct_22,2023-10-01,2023-10-31
2,nov_22,2023-11-01,2023-11-30
3,dec_22,2023-12-01,2023-12-31
4,jan_23,2024-01-01,2024-01-31


In [15]:
# Construct the connection string for connecting to the SQL Server database
connection_string = "DRIVER={SQL SERVER}; SERVER=" + SERVER  + "; DATABASE=" + DATABASE + "; UID=" + UID + "; PWD=" + PWD + ";"

# Establish a connection to the SQL Server database using the constructed connection string
connection = odbc.connect(connection_string)

# Iterate over each row in the renewals DataFrame to process data for each renewal period
for index, renewal in renewals_df.iterrows():
    # Extract the renewal month name, start date, and end date from the current row
    renewal_name = renewal['renewal_month_name'] 
    renewal_start = renewal['renewal_start']
    renewal_end = renewal['renewal_end']
    
    # Define a SQL query to retrieve membership and renewal data for the given renewal period
    query_renewal = f"""
    declare @CurrentDate datetime = '{renewal_start}',
              @FutureDate datetime = '{renewal_end}';
       -- Get all members as of Current Date
      with aba_members as (
      select od.ship_master_customer_id as Customer_ID,
            od.CYCLE_BEGIN_DATE, od.CYCLE_END_DATE, od.PRODUCT_CODE, od.ORDER_NO, od.ORDER_LINE_NO, od.GRACE_DATE,
            dense_rank() over (partition by od.ship_master_customer_id order by od.cycle_begin_date desc, od.order_no desc) the_rank
      from   order_detail od (nolock) join order_master om (nolock) on od.order_no = om.order_no
            join mbr_product mp on od.product_id = mp.product_id
               and mp.level1 = 'ABA'
      where  om.order_status_code = 'A'
      and    od.line_status_code = 'A'
      and    od.fulfill_status_code in ('A','E','G','T','X')
      and    @CurrentDate between cast(od.cycle_begin_date as date) and cast(od.grace_date as date)
      ),
      -- Get count of any prior orders where the cycle begin date of the current order is greater than the end date of prior orders
      prior_membership_orders as (
      select a.Customer_ID,
            count(distinct od.order_no + str(od.order_line_no)) order_count
      from   order_detail od (nolock) join order_master om (nolock) on od.order_no = om.order_no
            join mbr_product mp on od.product_id = mp.product_id
               and mp.level1 = 'ABA'
            join aba_members a on od.ship_master_customer_id = a.Customer_ID
      where  om.order_status_code = 'A'
      and    od.line_status_code = 'A'
      and    od.fulfill_status_code in ('A','E','G','T','X')
      and    od.Cycle_End_Date < a.Cycle_Begin_Date
      group by a.Customer_ID),
      prior_membership_order_count as (
      select a.Customer_ID,
            isnull(p.Order_Count,0) Order_Count
      from   aba_members a left join prior_membership_orders p on a.Customer_ID = p.Customer_ID)
      ,
      -- Identify future members based on future orders
      future_members as (
      select distinct od.ship_master_customer_id as Customer_ID
      from   order_detail od (nolock) join order_master om (nolock) on od.order_no = om.order_no
            join mbr_product mp on od.product_id = mp.product_id
               and mp.level1 = 'ABA'
      where  om.order_status_code = 'A'
      and    od.line_status_code = 'A'
      and    od.fulfill_status_code in ('A','E','G','T','X')
      and    @CurrentDate < cast(od.cycle_begin_date as date)  -- Take future order
      and    @FutureDate between cast(od.cycle_begin_date as date) and cast(od.grace_date as date)),
      -- Identify members up for renewal
      renewals as (
      select distinct od.ship_master_customer_id as Customer_ID
      from   order_detail od (nolock) join order_master om (nolock) on od.order_no = om.order_no
            join mbr_product mp on od.product_id = mp.product_id
               and mp.level1 = 'ABA'
      where  om.order_status_code = 'A'
      and    od.line_status_code = 'A'
      and    od.fulfill_status_code in ('A','E','G','T','X')
      and    eomonth(cast(od.cycle_end_date as date)) >= eomonth(@CurrentDate)),
      -- Combine all data into the main dataset
      raw_data as (
      select a.Customer_ID,
            a.Cycle_Begin_Date, a.CYCLE_END_DATE, a.product_code, a.order_no, a.order_line_no, a.grace_date,
            a.the_rank,
            (case when r.Customer_ID is not null then 1 else 0 end) Member_Renewal_Indicator,
            (case when f.Customer_ID is not null then 1 else 0 end) Member_Renewed_Indicator
      from   aba_members a left join renewals r on a.Customer_ID = r.Customer_ID       
            left join future_members f on r.Customer_ID = f.Customer_ID
      where	a.PRODUCT_CODE not in ('ABA-IN-STUDENT', 'ABA-IN-PSTUDENT', 'ABA-IN-ASSOC', 'ABA-IN-ASTUASSOC', 'ABA-IN-NCASSOC')
      and a.the_rank= 1
      )
      select distinct rd.*,
            mji.Earliest_Begin_Date, p.order_count
      from   raw_data rd left join prior_membership_order_count p on rd.Customer_ID = p.Customer_ID
            join usr_cus_member_join_info mji on rd.Customer_ID = mji.Master_Customer_ID
               and mji.Mbr_Product_Code = 'ABA'
      where  1=1
      and rd.the_rank= 1
      --Member_Renewal_Indicator = 1
      --and    Member_Renewed_Indicator = 1 -- Did not renew
      """

    # Execute the SQL query and load the results into a pandas DataFrame
    df_renewal = pd.read_sql(query_renewal, connection)
    
    # Store the resulting DataFrame in the global namespace with a name based on the renewal month
    globals()[f"renewal_{renewal_name}"] = df_renewal

C:\Users\SinghS\AppData\Local\Temp\1\ipykernel_21224\3712456600.py:94: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_renewal = pd.read_sql(query_renewal, connection)


In [16]:
# Create a list of month names corresponding to the cohort periods
as_of_month_name = ["sep_22", "oct_22", "nov_22", "dec_22", "jan_23", "feb_23", "mar_23", 
               "apr_23", "may_23", "jun_23", "jul_23", "aug_23"]

# Generate a range of 12 monthly start dates beginning from September 1, 2023
# 'freq="MS"' ensures the dates are the first day of each month
as_of_start_months = pd.date_range("2023-09-01", periods=12, freq="MS")

# Initialize an empty list to store dictionaries containing cohort start, end, and demographic dates
as_ofs = []

# Loop through each start month in the generated date range
for as_of_month in as_of_start_months:
    # Calculate the start date of the cohort as one day before the current month
    as_of_start = as_of_month + pd.DateOffset(days=-1)
    # Calculate the end date of the cohort as the last day of the current month
    as_of_end = as_of_month + pd.DateOffset(months=1, days=-1)  # Add 1 month, subtract 1 day
    # Calculate the demographic date as one year and one month before the current month, minus one day
    demographic_date = as_of_month + pd.DateOffset(years=-1, months=1, days=-1)
    
    # Append a dictionary with the calculated dates to the list
    as_ofs.append({'as_of_start_date': as_of_start, 'as_of_end_date': as_of_end, 'demographic_date': demographic_date})

# Convert the list of dictionaries into a DataFrame for easier manipulation and visualization
as_ofs_df = pd.DataFrame(as_ofs)

# Add the corresponding month names to the DataFrame
as_ofs_df['as_of_month_name'] = as_of_month_name

# Format the 'as_of_start_date', 'as_of_end_date', and 'demographic_date' columns as strings in 'YYYY-MM-DD' format
as_ofs_df['as_of_start_date'] = as_ofs_df['as_of_start_date'].dt.strftime('%Y-%m-%d')
as_ofs_df['as_of_end_date'] = as_ofs_df['as_of_end_date'].dt.strftime('%Y-%m-%d')
as_ofs_df['demographic_date'] = as_ofs_df['demographic_date'].dt.strftime('%Y-%m-%d')

# Reorder the columns to display month name, start date, end date, and demographic date in the desired order
as_ofs_df = as_ofs_df[['as_of_month_name', 'as_of_start_date', 'as_of_end_date', 'demographic_date']]

# Display the DataFrame to verify the results
as_ofs_df

,as_of_month_name,as_of_start_date,as_of_end_date,demographic_date
0,sep_22,2023-08-31,2023-09-30,2022-09-30
1,oct_22,2023-09-30,2023-10-31,2022-10-31
2,nov_22,2023-10-31,2023-11-30,2022-11-30
3,dec_22,2023-11-30,2023-12-31,2022-12-31
4,jan_23,2023-12-31,2024-01-31,2023-01-31
5,feb_23,2024-01-31,2024-02-29,2023-02-28
6,mar_23,2024-02-29,2024-03-31,2023-03-31
7,apr_23,2024-03-31,2024-04-30,2023-04-30
8,may_23,2024-04-30,2024-05-31,2023-05-31
9,jun_23,2024-05-31,2024-06-30,2023-06-30


In [17]:
connection_string = "DRIVER={SQL SERVER}; SERVER=" + SERVER  + "; DATABASE=" + DATABASE + "; UID=" + UID + "; PWD=" + PWD + ";"
connection = odbc.connect(connection_string)

# Define the query template with placeholders for the dates
query_demographic = """
SELECT
    ORDER_DETAIL.SHIP_MASTER_CUSTOMER_ID customer_id,
    MBR_ORDER_DETAIL.DUES_REQUIRED_SECTION_COUNT,
    MBR_ORDER_DETAIL.NO_CHARGE_SECTION_COUNT,
    MBR_ORDER_DETAIL.AUTO_ENROLL_SECTION_COUNT,
    SHIP_CUSTOMER.GENDER_CODE,
    SHIP_CUSTOMER.ETHNICITY_CODE,
    ABA_CUS_DEMOGRAPHIC_ABASET_VW.ABASET_CODE_DESCR,
    CAST(USR_ABA_MEMBER_JOIN_INFO.MOST_RECENT_ADD_DATE AS DATE) MOST_RECENT_ADD_DATE,
    CAST(SHIP_CUSTOMER.BIRTH_DATE AS DATE) DOB,
    CAST(ORDER_DETAIL.CYCLE_BEGIN_DATE AS DATE) CYCLE_BEGIN_DATE,
    CAST(ORDER_DETAIL.CYCLE_END_DATE AS DATE) CYCLE_END_DATE,
    CAST(ORDER_DETAIL.GRACE_DATE AS DATE) GRACE_DATE,
    (CASE WHEN (MBR_ORDER_DETAIL_PAYMENT_MESRS.ORDER_REVENUE_FY_MONTHS) = 0 THEN 'N'
          ELSE (MBR_ORDER_DETAIL.PAID_STATUS_CODE)
    END) PAYMENT_STATUS,
    CAST(ABA_MBR_AsOfDateFY_OrderSelction1.AS_OF_DATE_FY AS DATE) AS_OF_DATE,
    ABA_CUS_DISABILITY_INDICATOR_VW.DISABILITY_INDICATOR,
    ABA_CUS_DEMOGRAPHICS_SEXUAL_ORIEN_VW.DESCR,
    ABA_CUS_DEMOGRAPHIC_ABASET_VW.ABASET_SUBCODE_DESCR,
    SHIP_CUS_ADDRESS.STATE
FROM
    CUS_ADDRESS SHIP_CUS_ADDRESS
    INNER JOIN CUS_ADDRESS_DETAIL SHIP_CUS_ADDRESS_DETAIL 
        ON (SHIP_CUS_ADDRESS.CUS_ADDRESS_ID = SHIP_CUS_ADDRESS_DETAIL.CUS_ADDRESS_ID
        AND SHIP_CUS_ADDRESS.MASTER_CUSTOMER_ID = SHIP_CUS_ADDRESS_DETAIL.MASTER_CUSTOMER_ID
        AND SHIP_CUS_ADDRESS.SUB_CUSTOMER_ID = SHIP_CUS_ADDRESS_DETAIL.SUB_CUSTOMER_ID)
    INNER JOIN ORDER_DETAIL 
        ON (ORDER_DETAIL.SUBSYSTEM = 'MBR' 
        AND SHIP_CUS_ADDRESS_DETAIL.MASTER_CUSTOMER_ID = ORDER_DETAIL.SHIP_MASTER_CUSTOMER_ID
        AND SHIP_CUS_ADDRESS_DETAIL.SUB_CUSTOMER_ID = ORDER_DETAIL.SHIP_SUB_CUSTOMER_ID
        AND SHIP_CUS_ADDRESS_DETAIL.CUS_ADDRESS_ID = ORDER_DETAIL.SHIP_ADDRESS_ID 
        AND SHIP_CUS_ADDRESS_DETAIL.ADDRESS_TYPE_CODE = 
            (SELECT MAX(ADDRESS_TYPE_CODE) 
             FROM CUS_ADDRESS_DETAIL CAD1 (NOLOCK) 
             WHERE CAD1.MASTER_CUSTOMER_ID = ORDER_DETAIL.SHIP_MASTER_CUSTOMER_ID 
             AND CAD1.CUS_ADDRESS_ID = ORDER_DETAIL.SHIP_ADDRESS_ID)
        AND ORDER_DETAIL.SUBSYSTEM = 'MBR')
    INNER JOIN CUSTOMER SHIP_CUSTOMER 
        ON (ORDER_DETAIL.SHIP_MASTER_CUSTOMER_ID = SHIP_CUSTOMER.MASTER_CUSTOMER_ID 
        AND ORDER_DETAIL.SHIP_SUB_CUSTOMER_ID = SHIP_CUSTOMER.SUB_CUSTOMER_ID)
    LEFT OUTER JOIN ABA_CUS_DEMOGRAPHIC_ABASET_VW 
        ON (ABA_CUS_DEMOGRAPHIC_ABASET_VW.MASTER_CUSTOMER_ID = SHIP_CUSTOMER.MASTER_CUSTOMER_ID)
    LEFT OUTER JOIN USR_ABA_MEMBER_JOIN_INFO 
        ON (USR_ABA_MEMBER_JOIN_INFO.MASTER_CUSTOMER_ID = SHIP_CUSTOMER.MASTER_CUSTOMER_ID 
        AND USR_ABA_MEMBER_JOIN_INFO.PRODUCT_CODE = 'ABA')
    LEFT OUTER JOIN ABA_CUS_DEMOGRAPHICS_SEXUAL_ORIEN_VW 
        ON (ABA_CUS_DEMOGRAPHICS_SEXUAL_ORIEN_VW.MASTER_CUSTOMER_ID = SHIP_CUSTOMER.MASTER_CUSTOMER_ID)
    LEFT OUTER JOIN ABA_CUS_DISABILITY_INDICATOR_VW 
        ON (ABA_CUS_DISABILITY_INDICATOR_VW.MASTER_CUSTOMER_ID = SHIP_CUSTOMER.MASTER_CUSTOMER_ID)
    INNER JOIN USR_ABA_MBR_ORDER_DETAIL MBR_ORDER_DETAIL 
        ON (ORDER_DETAIL.ORDER_NO = MBR_ORDER_DETAIL.ORDER_NO 
        AND ORDER_DETAIL.ORDER_LINE_NO = MBR_ORDER_DETAIL.ORDER_LINE_NO)
    INNER JOIN USR_ABA_MBR_ORDER_DETAIL_PAYMENT MBR_ORDER_DETAIL_PAYMENT_MESRS 
        ON (MBR_ORDER_DETAIL.ORDER_NO = MBR_ORDER_DETAIL_PAYMENT_MESRS.ORDER_NO 
        AND MBR_ORDER_DETAIL.ORDER_LINE_NO = MBR_ORDER_DETAIL_PAYMENT_MESRS.ORDER_LINE_NO)
    INNER JOIN (
        SELECT q2.AS_OF_DATE AS AS_OF_DATE_FY,
               q2.FISCAL_YEAR,
               q2.SHIP_MASTER_CUSTOMER_ID,
               q2.ORDER_NO,
               q2.ORDER_LINE_NO,
               q2.HIGHEST_PAID_AMOUNT_DATE,
               q2.ORDER_WEIGHT,
               q2.PAYMENT_WEIGHT,
               q2.THE_RANK
        FROM (
            SELECT q1.*, 
                   RANK() OVER (PARTITION BY ship_master_customer_id 
                                ORDER BY (order_weight + payment_weight + Order_Line_status_weight + Order_Line_Fulfill_status_weight), 
                                         cycle_end_date DESC, order_no DESC, adddate DESC) AS the_rank
            FROM (
                SELECT '{demographic_date}' AS as_of_date,
                       xdp.FISCAL_YEAR,
                       od.ship_master_customer_id,
                       od.order_no,
                       od.order_line_no,
                       od.product_code,
                       xd.highest_paid_amount_date,
                       od.cycle_begin_date,
                       od.cycle_end_date,
                       od.line_status_code,
                       od.fulfill_status_code,
                       od.adddate,
                       CASE 
                           WHEN (xd.REPORTING_LEVEL3 = 'STUDENT' AND ('{demographic_date}' BETWEEN od.cycle_begin_date AND od.cycle_end_date)) THEN 15
                           WHEN (xd.REPORTING_LEVEL3 = 'STUDENT') THEN 20
                           WHEN ('{demographic_date}' BETWEEN od.cycle_begin_date AND od.cycle_end_date) AND xd.REPORTING_LEVEL3 != 'STUDENT' THEN 0
                           WHEN '{demographic_date}' > od.cycle_end_date THEN 5
                           ELSE 10
                       END AS order_weight,
                       CASE 
                           WHEN ('{demographic_date}' >= ISNULL(xd.Highest_Paid_Amount_Date, '12/31/2099')) THEN 5
                           WHEN ('{demographic_date}' < ISNULL(xd.Highest_Paid_Amount_Date, '12/31/2099')) THEN 15
                           ELSE 20
                       END AS payment_weight,
                       CASE
                           WHEN OD.LINE_STATUS_CODE = 'A' AND OM.ORDER_STATUS_CODE = 'A' THEN 0
                           WHEN OD.LINE_STATUS_CODE = 'C' OR OM.ORDER_STATUS_CODE = 'C' THEN 4
                           WHEN OD.LINE_STATUS_CODE = 'P' OR OM.ORDER_STATUS_CODE = 'P' THEN 2
                           ELSE 6
                       END AS Order_Line_status_weight,
                       CASE 
                           WHEN OD.FULFILL_STATUS_CODE = 'A' THEN 0
                           WHEN OD.FULFILL_STATUS_CODE IN ('E', 'G', 'T') THEN 1
                           WHEN OD.FULFILL_STATUS_CODE = 'S' THEN 4
                           WHEN OD.FULFILL_STATUS_CODE = 'X' THEN 5
                           WHEN OD.FULFILL_STATUS_CODE = 'H' THEN 6
                           ELSE 8
                       END AS Order_Line_Fulfill_status_weight
                FROM usr_aba_mbr_order_detail xd (nolock)
                JOIN order_detail od (nolock) ON xd.order_no = od.order_no AND xd.order_line_no = od.order_line_no
                JOIN usr_aba_mbr_order_detail_payment xdp (nolock) ON xd.order_no = xdp.order_no AND xd.order_line_no = xdp.order_line_no
                JOIN ORDER_MASTER OM (NOLOCK) ON (OD.ORDER_NO = OM.ORDER_NO)
                WHERE xd.aba_section_indicator = 1
                AND xdp.fiscal_year = 2023
                AND xd.highest_PAID_AMOUNT_DATE <= '{demographic_date}'
                AND (CASE WHEN (XDP.ORDER_REVENUE_FY_MONTHS) = 0 THEN 'N'
                          ELSE (XD.PAID_STATUS_CODE)
                 END) IN ('F', 'P')
            ) AS q1
        ) AS q2
        WHERE 1=1
        AND the_rank = 1
    ) ABA_MBR_AsOfDateFY_OrderSelction1 
        ON (MBR_ORDER_DETAIL_PAYMENT_MESRS.ORDER_NO = ABA_MBR_AsOfDateFY_OrderSelction1.ORDER_NO 
        AND MBR_ORDER_DETAIL_PAYMENT_MESRS.ORDER_LINE_NO = ABA_MBR_AsOfDateFY_OrderSelction1.ORDER_LINE_NO 
        AND MBR_ORDER_DETAIL_PAYMENT_MESRS.FISCAL_YEAR = ABA_MBR_AsOfDateFY_OrderSelction1.FISCAL_YEAR)
WHERE
    MBR_ORDER_DETAIL.ABA_SECTION_INDICATOR IN (1)
    AND ISNULL(MBR_ORDER_DETAIL.REPORTING_LEVEL1, 'Unknown') = 'Indiv'
    AND ISNULL(MBR_ORDER_DETAIL.REPORTING_LEVEL3, 'Unknown') = 'Lawyer'
    AND ISNULL(MBR_ORDER_DETAIL.NMM_DUES_CLASS, 'Unknown') = '0 to 4 Years'
    AND CAST(ORDER_DETAIL.GRACE_DATE AS DATE) > CAST(ABA_MBR_AsOfDateFY_OrderSelction1.AS_OF_DATE_FY AS DATE)
    AND CAST(ORDER_DETAIL.CYCLE_END_DATE AS DATE) BETWEEN '{as_of_start_date}' AND '{as_of_end_date}'
"""

In [18]:
# Create the 12 DataFrames with dates coming from 'as_ofs_df'
for idx, row in as_ofs_df.iterrows():
    # Extract the date variables for each row in the DataFrame
    as_of_start_date = row['as_of_start_date']
    as_of_end_date = row['as_of_end_date']
    demographic_date = row['demographic_date']
    
    # Format the query with the current dates
    query = query_demographic.format(as_of_start_date=as_of_start_date,
                                      as_of_end_date=as_of_end_date,
                                      demographic_date=demographic_date)
    
    # Execute the query and fetch the results
    df_result = pd.read_sql(query, connection)
    
    # Create a unique variable name for each DataFrame
    df_name = f"demographic_{row['as_of_month_name']}_df"
    
    # Store the result in a new global variable using globals()
    globals()[df_name] = df_result

C:\Users\SinghS\AppData\Local\Temp\1\ipykernel_21224\3892279910.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, connection)


In [19]:
connection.close()

### Merge the Data Frames

In [20]:
# Get a list of all DataFrames in the global namespace
dataframes = [var for var in globals() if isinstance(globals()[var], pd.DataFrame)]

# Print the list of DataFrame names
print(dataframes)

['_', '__', '___', 'j_df', 'monthly_df', 'jan_23_df', 'feb_23_df', 'mar_23_df', 'apr_23_df', 'may_23_df', 'jun_23_df', 'jul_23_df', 'aug_23_df', 'sep_22_df', 'oct_22_df', 'nov_22_df', 'dec_22_df', 'cohorts_df', '_9', 'df_cohort', 'order_sep_22', 'order_oct_22', 'order_nov_22', 'order_dec_22', 'order_jan_23', 'order_feb_23', 'order_mar_23', 'order_apr_23', 'order_may_23', 'order_jun_23', 'order_jul_23', 'order_aug_23', 'df', 'pivot_df', 'sep_22_pivot', 'oct_22_pivot', 'nov_22_pivot', 'dec_22_pivot', 'jan_23_pivot', 'feb_23_pivot', 'mar_23_pivot', 'apr_23_pivot', 'may_23_pivot', 'jun_23_pivot', 'jul_23_pivot', 'aug_23_pivot', 'duplicates', 'renewals_df', '_14', 'df_renewal', 'renewal_sep_22', 'renewal_oct_22', 'renewal_nov_22', 'renewal_dec_22', 'renewal_jan_23', 'renewal_feb_23', 'renewal_mar_23', 'renewal_apr_23', 'renewal_may_23', 'renewal_jun_23', 'renewal_jul_23', 'renewal_aug_23', 'as_ofs_df', '_16', 'df_result', 'demographic_sep_22_df', 'demographic_oct_22_df', 'demographic_nov_22

In [21]:
# Create a list of DataFrames, each representing renewal data for a specific month
renewal_dataframes = [renewal_sep_22, renewal_oct_22, renewal_nov_22, renewal_dec_22, 
                      renewal_jan_23, renewal_feb_23, renewal_mar_23, renewal_apr_23, 
                      renewal_may_23, renewal_jun_23, renewal_jul_23, renewal_aug_23]

# Iterate over each DataFrame in the list
for df in renewal_dataframes:
    # Iterate over each column in the current DataFrame
    for col in df.columns:
        # Rename the column to lowercase for consistency
        df.rename(columns={col: col.lower()}, inplace=True)

In [22]:
# Define a list of month identifiers corresponding to the data to be processed
months = ['sep_22', 'oct_22', 'nov_22', 'dec_22', 'jan_23', 'feb_23', 'mar_23', 
          'apr_23', 'may_23', 'jun_23', 'jul_23', 'aug_23']

# Loop through each month in the list to dynamically process and merge DataFrames
for month in months:
    # Dynamically construct the names of the DataFrames for the current month
    demographic_df_name = f'demographic_{month}_df'  # Name of the demographic DataFrame for the month
    renewal_df_name = f'renewal_{month}'            # Name of the renewal DataFrame for the month
    pivot_df_name = f'{month}_pivot'                # Name of the pivot DataFrame for the month
    month_specific_df_name = f'{month}_df'          # Name of the month-specific DataFrame

    # Use the globals() function to fetch the DataFrames by their dynamically constructed names
    demographic_df = globals()[demographic_df_name]  # Retrieve the demographic DataFrame
    renewal_df = globals()[renewal_df_name]          # Retrieve the renewal DataFrame
    pivot_df = globals()[pivot_df_name]              # Retrieve the pivot DataFrame
    month_specific_df = globals()[month_specific_df_name]  # Retrieve the month-specific DataFrame

    # Perform a series of left joins to merge the DataFrames on the 'customer_id' column
    merged_df = pd.merge(demographic_df, renewal_df, on='customer_id', how='left')  # Merge demographic and renewal data
    merged_df = pd.merge(merged_df, pivot_df, on='customer_id', how='left')         # Merge with pivot data
    merged_df = pd.merge(merged_df, month_specific_df, on='customer_id', how='left')  # Merge with month-specific data

    # Save the resulting merged DataFrame as a global variable with a dynamically constructed name
    globals()[f'merged_{month}_df'] = merged_df  # Store the merged DataFrame for the current month

In [ ]:
# List of DataFrames for each month from September 2022 to August 2023
merged_dfs = [ merged_sep_22_df, merged_oct_22_df, merged_nov_22_df, merged_dec_22_df,
              merged_jan_23_df, merged_feb_23_df, merged_mar_23_df, merged_apr_23_df,
              merged_may_23_df, merged_jun_23_df, merged_jul_23_df, merged_aug_23_df]

# Loop through each DataFrame in the list to add the 'age' column
for df in merged_dfs:
    # Convert 'DOB' (Date of Birth) column to datetime format for accurate calculations
    df['DOB'] = pd.to_datetime(df['DOB'])
    # Convert 'CYCLE_END_DATE' column to datetime format for accurate calculations
    df['CYCLE_END_DATE'] = pd.to_datetime(df['CYCLE_END_DATE'])
    
    # Calculate the age in years by finding the difference between 'CYCLE_END_DATE' and 'DOB' in days,
    # dividing by 365 to convert to years, and rounding to the nearest whole number
    df['age'] = round((df['CYCLE_END_DATE'] - df['DOB']).dt.days / 365)

In [32]:
# Combine multiple DataFrames into a single DataFrame
# The DataFrames being concatenated represent merged data for each month from September 2022 to August 2023
renewal_app_df = pd.concat([
    merged_sep_22_df,  # DataFrame for September 2022
    merged_oct_22_df,  # DataFrame for October 2022
    merged_nov_22_df,  # DataFrame for November 2022
    merged_dec_22_df,  # DataFrame for December 2022
    merged_jan_23_df,  # DataFrame for January 2023
    merged_feb_23_df,  # DataFrame for February 2023
    merged_mar_23_df,  # DataFrame for March 2023
    merged_apr_23_df,  # DataFrame for April 2023
    merged_may_23_df,  # DataFrame for May 2023
    merged_jun_23_df,  # DataFrame for June 2023
    merged_jul_23_df,  # DataFrame for July 2023
    merged_aug_23_df   # DataFrame for August 2023
], axis=0, ignore_index=True, sort=False).fillna(0)  # Concatenate along rows, reset index, and fill NaN values with 0

# Display a summary of the resulting DataFrame
# This includes information about the DataFrame's structure, such as column names, data types, and memory usage
renewal_app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8834 entries, 0 to 8833
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   customer_id                  8834 non-null   object        
 1   DUES_REQUIRED_SECTION_COUNT  8834 non-null   int64         
 2   NO_CHARGE_SECTION_COUNT      8834 non-null   int64         
 3   AUTO_ENROLL_SECTION_COUNT    8834 non-null   int64         
 4   GENDER_CODE                  8834 non-null   object        
 5   ETHNICITY_CODE               8834 non-null   object        
 6   ABASET_CODE_DESCR            8834 non-null   object        
 7   MOST_RECENT_ADD_DATE         8834 non-null   object        
 8   DOB                          8834 non-null   object        
 9   CYCLE_BEGIN_DATE             8834 non-null   object        
 10  CYCLE_END_DATE               8834 non-null   datetime64[ns]
 11  GRACE_DATE                   8834 non-null 

In [33]:
# Export the combined DataFrame 'renewal_app_df' to a CSV file
# The file will be saved in the './_data/' directory with the name 'renewal_app_df.csv'
renewal_app_df.to_csv('./_data/renewal_app_df.csv', index=False) # 'index=False' ensures that the DataFrame index is not written to the CSV file